<a href="https://colab.research.google.com/github/nugzar/mics-w207/blob/master/Nugzar_Nebieridze_p3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 3: Poisonous Mushrooms

In this project, you'll investigate properties of mushrooms. This classic dataset contains over 8000 observations, where each mushroom is described by a variety of features like color, odor, etc., and the target variable is an indicator for whether the mushroom is poisonous. Since all the observations are categorical, I've binarized the feature space. Look at the feature_names below to see all 126 binary names.

You'll start by running PCA to reduce the dimensionality from 126 down to 2 so that you can easily visualize the data. In general, PCA is very useful for visualization (though sklearn.manifold.tsne is known to produce better visualizations). Recall that PCA is a linear transformation. The 1st projected dimension is the linear combination of all 126 original features that captures as much of the variance in the data as possible. The 2nd projected dimension is the linear combination of all 126 original features that captures as much of the remaining variance as possible. The idea of dense low dimensional representations is crucial to machine learning!

Once you've projected the data to 2 dimensions, you'll experiment with clustering using KMeans and density estimation with Gaussian Mixture Models. Finally, you'll train a classifier by fitting a GMM for the positive class and a GMM for the negative class, and perform inference by comparing the probabilities output by each model.

As always, you're welcome to work on the project in groups and discuss ideas on the course wall, but please prepare your own write-up and write your own code.

In [0]:
%matplotlib inline

import urllib.request as urllib2 # For python3
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from matplotlib.colors import LogNorm

In [0]:
MUSHROOM_DATA = 'https://raw.githubusercontent.com/UCB-MIDS/207-Applied-Machine-Learning/master/Data/mushroom.data'
MUSHROOM_MAP = 'https://raw.githubusercontent.com/UCB-MIDS/207-Applied-Machine-Learning/master/Data/mushroom.map'

Load feature names.

In [0]:
feature_names = []

for line in urllib2.urlopen(MUSHROOM_MAP):
    [index, name, junk] = line.decode('utf-8').split()
    feature_names.append(name)

print('Loaded feature names: ', len(feature_names))

Load data. The data is sparse in the input file, but there aren't too many features, so we'll use a dense representation, which is supported by all sklearn objects.

In [0]:
X, Y = [], []

for line in urllib2.urlopen(MUSHROOM_DATA):
    items = line.decode('utf-8').split()
    Y.append(int(items.pop(0)))
    x = np.zeros(len(feature_names))
    for item in items:
        feature = int(str(item).split(':')[0])
        x[feature] = 1
    X.append(x)

# Convert these lists to numpy arrays.
X = np.array(X)
Y = np.array(Y)

# Split into train and test data.
train_data, train_labels = X[:7000], Y[:7000]
test_data, test_labels = X[7000:], Y[7000:]

# Check that the shapes look right.
print(train_data.shape, test_data.shape)

### Part 1:

Run Principal Components Analysis on the data. Show what fraction of the total variance in the training data is explained by the first k principal components, for k in [1, 50].

In [0]:
def P1():
  ### STUDENT START ###

  np.random.seed(0)
  
  model = PCA()
  model.fit_transform(train_data)

  # We need cumulative sums by components
  variances = model.explained_variance_ratio_.cumsum()

  # Printing variances by components [1,50]
  # The last row is the accuracy of 50 components
  for k in range(50):
    print ("k =", k + 1, " Variance =", variances[k])

  ### STUDENT END ###

P1()

### Part 2:

PCA can be very useful for visualizing data. Project the training data down to 2 dimensions and plot it. Show the positive (poisonous) cases in blue and the negative (non-poisonous) in red. Here's a reference for plotting: http://matplotlib.org/users/pyplot_tutorial.html

In [0]:
def P2():
  
  ### STUDENT START ###

  np.random.seed(0)
  
  model = PCA(n_components=2)
  results = model.fit_transform(train_data)

  plt.figure(figsize = (5,5))
  plt.axis([-3,3,-3,3])

  # when train_label == 1, then the mushroom is poisonous
  # when train_label == 0, then the mushroom is not poisonous

  plt.plot(results[:,0][train_labels==1], results[:,1][train_labels==1], 'bo', markersize=1)
  plt.plot(results[:,0][train_labels==0], results[:,1][train_labels==0], 'ro', markersize=1)

  plt.xlabel('Component 1')
  plt.ylabel('Component 2')
  plt.title("PCA components=2 data visualization")
  plt.legend(["Poisonous", "Non poisonous"]) 

  ### STUDENT END ###

P2()

### Part 3:

Run KMeans with [1,16] clusters over the 2d projected data. Mark each centroid cluster and plot a circle that goes through the most distant point assigned to each cluster.

In [0]:
def P3():

  ### STUDENT START ###

  np.random.seed(0)

  model = PCA(n_components=2)
  results = model.fit_transform(train_data)

  # Do not clearly understood if we need 16 different plots for each cluster 
  # in range or just the plot with 16 clusters. Lets draw all 16 plots

  for cluster in range(1,17):

    plt.figure(figsize = (10,10))
    plt.axis([-3,3,-3,3])
    plt.plot(results[:,0][train_labels==1], results[:,1][train_labels==1], 'bo', markersize=1)
    plt.plot(results[:,0][train_labels==0], results[:,1][train_labels==0], 'ro', markersize=1)

    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.title("PCA components=2, KMeans clusters=%d" % cluster)
    plt.legend(["Poisonous", "Non poisonous"]) 

    # Found a good samle of calculating the Diameter of the cluster
    # Will be using it
    # https://datascience.stackexchange.com/questions/32753/find-cluster-diameter-and-associated-cluster-points-with-kmeans-clustering-scik
  
    kmodel = KMeans(n_clusters=cluster).fit(results)
    kcenters = kmodel.cluster_centers_
    kpredictions = kmodel.predict(results)
    plt.scatter(kcenters[:, 0], kcenters[:, 1], c='black', s=30, zorder=3)
    ax = plt.gca()

    for ix in range(cluster):
      # Calculating radius
      radii = max([np.linalg.norm(np.subtract(i,kcenters[ix])) for i in zip(results[kpredictions == ix, 0],results[kpredictions == ix, 1])])
      ax.add_patch(plt.Circle(kcenters[ix],radii,fill=False,alpha=0.5))

  ### STUDENT END ###

P3()

### Part 4:

Fit a Gaussian Mixture Model for the positive examples in your 2d projected data. Plot the estimated density contours as shown here: http://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_pdf.html#example-mixture-plot-gmm-pdf-py. Vary the number of mixture components from 1-4 and the covariance matrix type ('spherical', 'diag', 'tied', 'full').

In [0]:
def P4():

  ### STUDENT START ###

  np.random.seed(0)
  
  model = PCA(n_components=2)
  results = model.fit_transform(train_data)
  positives = results[train_labels == 1]
  covar_types = ['spherical', 'diag', 'tied', 'full']
  
  for component in range(4):
    for covar_type in covar_types:

      gmm = GaussianMixture(n_components=component+1, covariance_type=covar_type).fit(positives)

      # Also using some sample code from the internet :)

      x = np.linspace(-4.0, 4.0)
      y = np.linspace(-4.0, 4.0)
      X, Y = np.meshgrid(x, y)
      XX = np.array([X.ravel(), Y.ravel()]).T
      Z = -gmm.score_samples(XX)
      Z = Z.reshape(X.shape)
      
      plt.figure(figsize = (10, 10))
      CS = plt.contour(X, Y, Z, norm=LogNorm(vmin=1.0, vmax=1000.0), levels=np.logspace(0, 3, 10))
      CB = plt.colorbar(CS, shrink=0.8, extend='both')
      plt.scatter(positives[:, 0], positives[:, 1], .8)
      plt.title("PCA components=2, GMM compoments=%d, Covariance = %s" % (component+1, covar_type))
      plt.show()

  ### STUDENT END ###

P4()

### Part 5:

Fit two 4-component full covariance GMMs, one for the positive examples and one for the negative examples in your 2d projected data. Predict the test examples by choosing the label for which the model gives a larger probability (use GMM.score). What is the accuracy?

In [0]:
def P5():

  ### STUDENT START ###

  np.random.seed(0)

  model = PCA(n_components=2)
  train_results = model.fit_transform(train_data)
  test_results = model.transform(test_data)

  train_positives = train_results[train_labels == 1]
  train_negatives = train_results[train_labels == 0]

  gmm_positives = GaussianMixture(n_components=4, covariance_type="full").fit(train_positives)
  gmm_negatives = GaussianMixture(n_components=4, covariance_type="full").fit(train_negatives)

  correct_positives = 0
  correct_negatives = 0

  for i in range(len(test_labels)):
    if ((gmm_positives.score([test_results[i]]) > gmm_negatives.score([test_results[i]])) and test_labels[i] == 1):
          correct_positives = correct_positives + 1

    if ((gmm_positives.score([test_results[i]]) < gmm_negatives.score([test_results[i]])) and test_labels[i] == 0):
          correct_negatives = correct_negatives + 1

  print('Accuracy of positives', correct_positives / len(test_labels[test_labels == 1]))
  print('Accuracy of negatives', correct_negatives / len(test_labels[test_labels == 0]))
  print('Total Accuracy', (correct_positives + correct_negatives) / len(test_labels))

  ### STUDENT END ###

P5()

### Part 6:

Ideally, we'd like a model that gives the best accuracy with the fewest parameters. Run a series of experiments to find the model that gives the best accuracy with no more than 50 parameters. For example, with 3 PCA components and 2-component diagonal covariance GMMs, you'd have:

( (3 mean vector + 3 covariance matrix) x 2 components ) x 2 classes = 24 parameters

You should vary the number of PCA components, the number of GMM components, and the covariance type.

In [0]:
def P6():

  ### STUDENT START ###

  np.random.seed(0)

  # Depending on covariance types, PCA and GMM components can take the ranges
  # PCA [1, 24] and GMM [1, 24] 

  # full = (n_pca*n_gmm + n_pca (n_pca + 1)/ 2 * n_gmm) * n_classes
  # diagonal = (n_pca*n_gmm + n_pca * n_gmm) * n_classes
  # spherical = (n_pca*n_gmm + n_gmm) * n_classes
  # tied = (n_pca*n_gmm + n_pca (n_pca + 1)/ 2) * n_classes

  # So, lets calculate the best accuracy for these ranges

  best_combination = ()
  best_accuracy = 0.

  print ("Best result will be printed at the end\n")

  for pca_component in range(1, 25):

    model = PCA(n_components=pca_component)
    train_results = model.fit_transform(train_data)
    test_results = model.transform(test_data)

    train_positives = train_results[train_labels == 1]
    train_negatives = train_results[train_labels == 0]

    for gmm_component in range(1, 25):

      for covar_type in ['spherical', 'diag', 'tied', 'full']:

        # Let's calculate the number of parameters and if it is larger than 50 
        # then continue
        parameters_count = 0

        if covar_type == 'spherical':
          parameters_count = (pca_component * gmm_component + gmm_component) * 2
        elif covar_type == 'diag':
          parameters_count = (pca_component * gmm_component + pca_component * gmm_component) * 2
        elif covar_type == 'tied':
          parameters_count = (pca_component * gmm_component + pca_component * (pca_component + 1) / 2) * 2
        elif covar_type == 'full':
          parameters_count = (pca_component * gmm_component + pca_component * (pca_component + 1) / 2 * gmm_component) * 2

        if parameters_count > 50:
          continue

        gmm_positives = GaussianMixture(n_components=gmm_component, covariance_type=covar_type).fit(train_positives)
        gmm_negatives = GaussianMixture(n_components=gmm_component, covariance_type=covar_type).fit(train_negatives)

        correct_predictions = 0

        for i in range(len(test_labels)):
          if ((gmm_positives.score([test_results[i]]) > gmm_negatives.score([test_results[i]])) and test_labels[i] == 1):
            correct_predictions = correct_predictions + 1

          if ((gmm_positives.score([test_results[i]]) < gmm_negatives.score([test_results[i]])) and test_labels[i] == 0):
            correct_predictions = correct_predictions + 1

        if best_accuracy < correct_predictions / len(test_labels):
          best_combination = (pca_component, gmm_component, covar_type, parameters_count, correct_predictions / len(test_labels))
          best_accuracy = correct_predictions / len(test_labels)

        print ("Processed: PCA = %d, GMM = (%d,%s), Parameters = %d, Accuracy = %f" % (pca_component, gmm_component, covar_type, parameters_count, correct_predictions / len(test_labels)))

  print ("\nBest result: PCA = %d, GMM = (%d,%s), Parameters = %d, Accuracy = %f" % best_combination)

  # This loop takes a while, so I am printing the results below
  # Best result: PCA = 7, GMM = (3,spherical), Parameters = 48, Accuracy = 0.973310

  ### STUDENT END ###

P6()